# Kernel Telemetry Analysis Automation

This notebook automates the process of executing malware samples in a controlled VM environment with kernel-level telemetry collection:
1. Restore VM to snapshot 31
2. Start the VM
3. Install kernel driver (install.bat)
4. Start GP driver service (sc start gp-driver)
5. Execute malware sample (already present in VM)
6. Terminate malware process
7. Process telemetry data (rename main.py.dll to main.py)
8. Run Python analysis script
9. Copy CSV results to shared folder

In [60]:
import os
import time
import subprocess
import itertools
import re
import shutil

In [61]:
# Create necessary directories
os.system('mkdir -p screenshots')
os.system('mkdir -p kernel-telemetry-results')

0

In [62]:
# VM configuration
vm_name = 'windows11-clone Clone'  # Updated VM name
snapshot_no = 34  # Updated snapshot number

# Command templates
vm_open_cmd = f'vboxmanage startvm "{vm_name}"'
run_cmd = f'vboxmanage guestcontrol "{vm_name}" run --username wineleven --password therealtreasure -- '
run_cmd += '"C:\\Windows\\System32\\cmd.exe" "/c" '
copyfrom_cmd = f'vboxmanage guestcontrol "{vm_name}" --username wineleven --password therealtreasure copyfrom '

# Paths and directories
output_folder = 'kernel-telemetry-results'
hafidz_dir = 'C:\\Users\\wineleven\\Documents\\Hafidz\\'
shared_folder_path = "C:\\Users\\wineleven\\Desktop\\Shared"
z_drive = "Z:"

In [63]:
# Utility functions

def tstamp():
    """Get current timestamp in readable format."""
    t = time.time()
    return time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(t)) + ("%.3f" % (t % 1))[1:]

def check_output_t(command):
    """Execute command and return output."""
    ret = subprocess.check_output(command, shell=True)
    return ret

def run_vm_command(command, wait_time=2):
    """Execute command in VM and wait for completion."""
    full_cmd = run_cmd + f'"{command}"'
    print(f"[{tstamp()}] Executing: {command}")
    try:
        result = subprocess.run(full_cmd, shell=True, capture_output=True, text=True, timeout=30)
        if result.returncode != 0:
            print(f"Warning: Command failed with return code {result.returncode}")
            print(f"Error: {result.stderr}")
        time.sleep(wait_time)
        return result
    except subprocess.TimeoutExpired:
        print(f"Command timed out: {command}")
        return None

In [64]:
# VM operations

def restore(snap_no):
    """Restore VM to specified snapshot."""
    print(f"[{tstamp()}] Restoring snapshot {snap_no}")
    check_output_t(f'vboxmanage snapshot "{vm_name}" restore "Snapshot {snap_no}"')
    print(f"[{tstamp()}] Restoring snapshot {snap_no} done")

def shutdown_and_restore(snap_no):
    """Power off VM and restore to snapshot."""
    print(f"[{tstamp()}] Powering off VM")
    try:
        check_output_t(f'vboxmanage controlvm "{vm_name}" poweroff')
        print(f"[{tstamp()}] Powering off VM done")
    except:
        print('VM (probably) already off')
    time.sleep(3)
    restore(snap_no)

def start_vm():
    """Start the VM."""
    print(f"[{tstamp()}] Starting up VM")
    check_output_t(vm_open_cmd)
    print(f"[{tstamp()}] Starting up VM done")

def take_screenshot(filename):
    """Take a screenshot of the VM."""
    print(f"[{tstamp()}] Taking screenshot")
    os.system(f'vboxmanage controlvm "{vm_name}" screenshotpng screenshots/{filename}.png')
    print(f"[{tstamp()}] Taking screenshot done")

In [65]:
# Kernel driver and telemetry operations

def install_kernel_driver():
    """Install kernel driver using install.bat."""
    print(f"[{tstamp()}] Installing kernel driver")
    # Change to Hafidz directory and run install.bat
    install_cmd = f"cd /d {hafidz_dir} && install.bat"
    run_vm_command(install_cmd, wait_time=5)
    print(f"[{tstamp()}] Kernel driver installation completed")

def start_gp_driver():
    """Start the GP driver service."""
    print(f"[{tstamp()}] Starting GP driver service")
    # Change to Hafidz directory and start service
    start_cmd = f"cd /d {hafidz_dir} && sc start gp-driver"
    run_vm_command(start_cmd, wait_time=3)
    print(f"[{tstamp()}] GP driver service started")
    
def start_ioctl():
    """Run IOCTL.exe after starting GP driver service."""
    print(f"[{tstamp()}] Starting IOCTL.exe")
    # Change to Hafidz directory and run IOCTL.exe
    ioctl_cmd = f"cd /d {hafidz_dir} && IOCTL.exe"
    run_vm_command(ioctl_cmd, wait_time=5)
    print(f"[{tstamp()}] IOCTL.exe started")
    
def process_alert_data():
    """Process alert data using IOCTL export-csv command."""
    print(f"[{tstamp()}] Processing alert data")
    
    # Generate timestamp for alerts filename
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    alerts_filename = f"alerts_{timestamp}.csv"
    
    # Change to Hafidz directory and export alerts to CSV
    alert_cmd = f'cd /d {hafidz_dir} && IOCTL.exe export-csv {alerts_filename} alerts'
    run_vm_command(alert_cmd, wait_time=5)
    
    print(f"[{tstamp()}] Alert data processing completed - saved as {alerts_filename}")

def process_telemetry_data():
    """Process telemetry data: rename dll, run python script, copy CSV files."""
    print(f"[{tstamp()}] Processing telemetry data")
    
    # Change to Hafidz directory and execute the sequence
    process_cmd = f"cd /d {hafidz_dir} && ren main.py.dll main.py && python main.py && copy *.csv {z_drive}"
    run_vm_command(process_cmd, wait_time=10)
    
    print(f"[{tstamp()}] Telemetry data processing completed")

def copy_csv_results():
    """Copy CSV results from VM to host."""
    print(f"[{tstamp()}] Copying CSV results to host")
    
    # Search for CSV files in the shared folder (Z: drive maps to shared folder)
    csv_search_cmd = f"dir {z_drive}\\*.csv /b"
    
    try:
        result = run_vm_command(csv_search_cmd)
        if result and result.stdout:
            csv_files = result.stdout.strip().split('\n')
            
            for csv_file in csv_files:
                csv_file = csv_file.strip()
                if csv_file and len(csv_file) > 0:  # Skip empty results
                    src_path = f"{z_drive}\\{csv_file}"
                    dst_path = f"{output_folder}/{csv_file}"
                    
                    # Copy file from VM to host
                    copy_cmd = f'{copyfrom_cmd} "{src_path}" "{dst_path}"'
                    os.system(copy_cmd)
                    print(f"[{tstamp()}] Copied {csv_file} to {output_folder}")
    except Exception as e:
        print(f"Error copying CSV files: {e}")
    
    print(f"[{tstamp()}] CSV results copying from vm to host completed")
    """Copy CSV results from sharedfolder to ScriptDataMalwareHafidz inside /home/uwuntu/Downloads"""
    print(f"[{tstamp()}] Copying CSV results from sharedfolder to ScriptDataMalwareHafidz")

    source_folder = "/home/uwuntu/Downloads/sharedfolder"
    destination_folder = "/home/uwuntu/Downloads/ScriptDataMalwareHafidz"

    try:
        # Pastikan folder tujuan ada
        os.makedirs(destination_folder, exist_ok=True)

        # Loop semua file CSV dan salin
        for filename in os.listdir(source_folder):
            if filename.lower().endswith(".csv"):
                src_path = os.path.join(source_folder, filename)
                dst_path = os.path.join(destination_folder, filename)

                shutil.copy2(src_path, dst_path)
                print(f"[{tstamp()}] Copied: {filename}")

    except Exception as e:
        print(f"[{tstamp()}] Error copying CSV files: {e}")

    print(f"[{tstamp()}] CSV results copying completed")

In [66]:
# Malware operations

def execute_malware(malware_path):
    print(f"[{tstamp()}] Executing malware sample: {malware_path}")

    full_cmd = [
        "vboxmanage", "guestcontrol", vm_name, "run",
        "--username", "wineleven",
        "--password", "therealtreasure",
        "--", "cmd.exe", "/c", "start", "", malware_path
    ]

    # Jalankan di background tanpa blocking
    subprocess.Popen(full_cmd)

    print(f"[{tstamp()}] Malware execution started")

def wait_malware(wait_time):
    """Wait for specified time with progress indicator."""
    print(f"[{tstamp()}] Waiting for {wait_time} seconds for malware execution")
    for i in range(wait_time):
        print('\r[' + '#' * (i+1) + '_' * (wait_time-i-1) + f'] {i+1}/{wait_time}s', end='')
        time.sleep(1)
    print('\r[' + '#' * wait_time + f'] {wait_time}/{wait_time}s - Complete!')

def terminate_malware(malware_name):
    """Terminate the malware process."""
    print(f"[{tstamp()}] Terminating malware: {malware_name}")
    
    # Kill process by executable name
    kill_cmd = f"taskkill /f /im {malware_name}"
    run_vm_command(kill_cmd, wait_time=2)
    
    # Also try to kill any suspicious processes
    suspicious_processes = ['*.exe', 'malware*', 'ransomware*']
    for process in suspicious_processes:
        try:
            kill_cmd = f"taskkill /f /im {process}"
            run_vm_command(kill_cmd, wait_time=1)
        except:
            pass  # Ignore errors for process that might not exist
    
    print(f"[{tstamp()}] Malware termination completed")

In [67]:
def run_kernel_telemetry_analysis(malware_path, malware_name, execution_time=60):
    """Run the complete kernel telemetry analysis workflow."""
    print(f"[{tstamp()}] Starting kernel telemetry analysis for {malware_path}")
    print(f"[{tstamp()}] Malware execution time: {execution_time} seconds")
    
    # Initialize timing
    t0 = time.time()
    
    try:
        # Step 1: Restore snapshot and start VM
        restore(snapshot_no)
        start_vm()
        time.sleep(5)  # Allow system to stabilize
        
        # Step 2: Install kernel driver
        install_kernel_driver()
        
        # Step 3: Start GP driver service
        start_gp_driver()
        
        # Step 4: Start IOCTL.exe
        start_ioctl()
        
        # Step 5: Execute malware (already present in VM)
        execute_malware(malware_path)
        
        # Step 6: Wait for malware execution
        wait_malware(execution_time)
        
        # Step 7: Take screenshot for evidence
        screenshot_name = f"{malware_name}_execution"
        take_screenshot(screenshot_name)
        
        # Step 8: Terminate malware
        terminate_malware(malware_name)
        
        # Step 9: Process alert data
        process_alert_data()
        
        # Step 10: Process telemetry data
        time.sleep(5)  # Allow system to settle after termination
        process_telemetry_data()
        
        # Step 11: Copy CSV results to host
        copy_csv_results()
        
        # Step 12: Take final screenshot
        final_screenshot = f"{malware_name}_final"
        take_screenshot(final_screenshot)
        
        # Step 13: Shutdown and restore VM
        time.sleep(3)
        shutdown_and_restore(snapshot_no)
        
    except Exception as e:
        print(f"Error during analysis: {e}")
        # Emergency cleanup
        try:
            terminate_malware(malware_name)
        except:
            pass
        shutdown_and_restore(snapshot_no)
    
    elapsed = time.time() - t0
    print(f'[{tstamp()}] Kernel telemetry analysis completed in {elapsed:.1f} seconds.\n')

## Run Analysis for a Single Malware Sample

Use this cell to analyze a single malware sample with kernel telemetry. The malware should already be present in the VM.

In [68]:
# Path to malware sample in VM and its executable name
malware_path = r"D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe"  # Full path in VM
malware_name = "0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe"  # Just the executable name for process killing

# Execution time in seconds (how long to let malware run)
execution_time = 5  # Default: 60 seconds

# Uncomment the line below to run analysis on a single sample
run_kernel_telemetry_analysis(malware_path, malware_name, execution_time)

[2025-05-22 14:21:23.866] Starting kernel telemetry analysis for D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe
[2025-05-22 14:21:23.866] Malware execution time: 5 seconds
[2025-05-22 14:21:23.866] Restoring snapshot 33


0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-22 14:21:24.415] Restoring snapshot 33 done
[2025-05-22 14:21:24.416] Starting up VM
[2025-05-22 14:21:32.128] Starting up VM done
[2025-05-22 14:21:37.128] Installing kernel driver
[2025-05-22 14:21:37.129] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && install.bat
[2025-05-22 14:21:42.507] Kernel driver installation completed
[2025-05-22 14:21:42.508] Starting GP driver service
[2025-05-22 14:21:42.508] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && sc start gp-driver
[2025-05-22 14:21:46.634] GP driver service started
[2025-05-22 14:21:46.634] Starting IOCTL.exe
[2025-05-22 14:21:46.634] Executing: cd /d C:\Users\wineleven\Documents\Hafidz\ && IOCTL.exe
Command timed out: cd /d C:\Users\wineleven\Documents\Hafidz\ && IOCTL.exe
[2025-05-22 14:22:16.666] IOCTL.exe started
[2025-05-22 14:22:16.666] Executing malware sample: D:\new\ransomware_high_confidence\avoslocker\0b1f19ba8740b10ed017671aab023228756a6864fb008bf23f3c606189bdcd98.exe
[2025-05-22 14:22:16.

0%...10%...20%...30%...40%...50%...60%...70%...80%...90%...100%


[2025-05-22 14:23:09.854] Powering off VM done
[2025-05-22 14:23:12.855] Restoring snapshot 33


0%...10%...20%...30%...40%...50%...

[2025-05-22 14:23:13.488] Restoring snapshot 33 done
[2025-05-22 14:23:13.488] Kernel telemetry analysis completed in 109.6 seconds.



60%...70%...80%...90%...100%


## Run Analysis for Multiple Samples

Use this cell to analyze multiple malware samples already present in the VM.

In [ ]:
import os
import time
import subprocess
from datetime import datetime

# =================================================================================================
# CONFIGURATION
# =================================================================================================

VM_NAME = 'windows11-clone Clone'
SNAPSHOT_NO = 34
EXECUTION_TIME_DEFAULT = 60
DELAY_BETWEEN_DEFAULT = 30

MALWARE_DIRECTORIES = [
    "D:\\new\\ransomware_high_confidence\\avaddon",
    "D:\\new\\ransomware_high_confidence\\avoslocker", 
    "D:\\new\\ransomware_high_confidence\\babuk",
    "D:\\new\\ransomware_high_confidence\\bianlian",
    "D:\\new\\ransomware_high_confidence\\blackbasta",
    "D:\\new\\ransomware_high_confidence\\blackbyte",
    "D:\\new\\ransomware_high_confidence\\blackcat",
    "D:\\new\\ransomware_high_confidence\\blackmatter",
    "D:\\new\\ransomware_high_confidence\\bluesky",
    "D:\\new\\ransomware_high_confidence\\clop",
    "D:\\new\\ransomware_high_confidence\\conti",
    "D:\\new\\ransomware_high_confidence\\darkside",
    "D:\\new\\ransomware_high_confidence\\dharma",
    "D:\\new\\ransomware_high_confidence\\doppelpaymer",
    "D:\\new\\ransomware_high_confidence\\exorcist",
    "D:\\new\\ransomware_high_confidence\\gandcrab",
    "D:\\new\\ransomware_high_confidence\\hive",
    "D:\\new\\ransomware_high_confidence\\holyghost",
    "D:\\new\\ransomware_high_confidence\\karma",
    "D:\\new\\ransomware_high_confidence\\lockbit",
    "D:\\new\\ransomware_high_confidence\\lorenz",
    "D:\\new\\ransomware_high_confidence\\makop",
    "D:\\new\\ransomware_high_confidence\\maui",
    "D:\\new\\ransomware_high_confidence\\maze",
    "D:\\new\\ransomware_high_confidence\\mountlocker",
    "D:\\new\\ransomware_high_confidence\\nefilim",
    "D:\\new\\ransomware_high_confidence\\netwalker",
    "D:\\new\\ransomware_high_confidence\\nightsky",
    "D:\\new\\ransomware_high_confidence\\phobos",
    "D:\\new\\ransomware_high_confidence\\playcrypt",
    "D:\\new\\ransomware_high_confidence\\pysa",
    "D:\\new\\ransomware_high_confidence\\quantum",
    "D:\\new\\ransomware_high_confidence\\ragnarok",
    "D:\\new\\ransomware_high_confidence\\ransomexx",
    "D:\\new\\ransomware_high_confidence\\revil",
    "D:\\new\\ransomware_high_confidence\\ryuk",
    "D:\\new\\ransomware_high_confidence\\stop",
    "D:\\new\\ransomware_high_confidence\\thanos",
    "D:\\new\\ransomware_high_confidence\\wastedlocker",
    "D:\\new\\ransomware_high_confidence\\zeppelin"
]

PRIORITY_FAMILIES = ['avoslocker', 'lockbit', 'ryuk', 'conti', 'revil', 'maze', 'clop']

# =================================================================================================
# HELPER FUNCTIONS
# =================================================================================================

def log_info(message):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {message}")

def log_success(message):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] ✅ {message}")

def log_error(message):
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] ❌ {message}")

def start_vm_clean():
    """Start VM dari snapshot bersih"""
    log_info("Starting VM...")
    restore(SNAPSHOT_NO)
    start_vm()
    time.sleep(15)
    log_success("VM started")

def shutdown_vm_clean():
    """Shutdown VM dan restore"""
    log_info("Shutting down VM...")
    shutdown_and_restore(SNAPSHOT_NO)
    log_success("VM shutdown complete")

def scan_vm_directory(directory):
    """Scan satu direktori di VM untuk file .exe"""
    family_name = os.path.basename(directory.replace('\\', '/'))
    files = []
    
    try:
        cmd = f'dir "{directory}\\*.exe" /b'
        result = run_vm_command(cmd, wait_time=5)
        
        if result and result.stdout:
            file_lines = result.stdout.strip().split('\n')
            for line in file_lines:
                line = line.strip()
                if line and line.endswith('.exe') and not 'File Not Found' in line:
                    files.append({
                        'family': family_name,
                        'filename': line,
                        'full_path': f"{directory}\\{line}"
                    })
    except:
        pass
    
    return files

# =================================================================================================
# MAIN FUNCTIONS
# =================================================================================================

def test_vm_connection():
    """Test koneksi VM"""
    print("\n🔧 TESTING VM CONNECTION...")
    print("=" * 50)
    
    try:
        start_vm_clean()
        
        # Test 3 direktori
        test_dirs = MALWARE_DIRECTORIES[:3]
        success_count = 0
        
        for test_dir in test_dirs:
            log_info(f"Testing: {os.path.basename(test_dir)}")
            result = run_vm_command(f'dir "{test_dir}" /b', wait_time=3)
            
            if result and result.stdout and 'cannot find' not in result.stdout.lower():
                log_success(f"✅ {os.path.basename(test_dir)} accessible")
                success_count += 1
            else:
                log_error(f"❌ {os.path.basename(test_dir)} not accessible")
        
        shutdown_vm_clean()
        
        if success_count > 0:
            log_success(f"VM CONNECTION TEST PASSED ({success_count}/{len(test_dirs)} directories OK)")
            return True
        else:
            log_error("VM CONNECTION TEST FAILED")
            return False
            
    except Exception as e:
        log_error(f"VM test failed: {e}")
        try:
            shutdown_vm_clean()
        except:
            pass
        return False

def scan_all_malware():
    """Scan semua malware di VM"""
    print("\n📁 SCANNING ALL MALWARE...")
    print("=" * 50)
    
    try:
        start_vm_clean()
        
        all_files = []
        family_stats = {}
        
        for directory in MALWARE_DIRECTORIES:
            family = os.path.basename(directory.replace('\\', '/'))
            log_info(f"Scanning {family}...")
            
            files = scan_vm_directory(directory)
            all_files.extend(files)
            family_stats[family] = len(files)
            
            if len(files) > 0:
                print(f"  {family}: {len(files)} files")
        
        shutdown_vm_clean()
        
        print(f"\n📊 SCAN RESULTS:")
        print(f"Total families: {len(family_stats)}")
        print(f"Total files: {len(all_files)}")
        
        print(f"\nTop families by file count:")
        sorted_families = sorted(family_stats.items(), key=lambda x: x[1], reverse=True)
        for family, count in sorted_families[:10]:
            if count > 0:
                print(f"  {family}: {count} files")
        
        return all_files
        
    except Exception as e:
        log_error(f"Scan failed: {e}")
        try:
            shutdown_vm_clean()
        except:
            pass
        return []

def run_batch_analysis(selected_files, execution_time=60):
    """Run analisis batch"""
    if not selected_files:
        log_error("No files selected for analysis")
        return
    
    print(f"\n🔬 STARTING BATCH ANALYSIS...")
    print("=" * 50)
    print(f"Total samples: {len(selected_files)}")
    print(f"Execution time per sample: {execution_time}s")
    
    successful = 0
    failed = 0
    failed_samples = []
    start_time = time.time()
    
    for idx, file_info in enumerate(selected_files, 1):
        family = file_info['family']
        filename = file_info['filename']
        full_path = file_info['full_path']
        
        print(f"\n[{idx}/{len(selected_files)}] {family}/{filename}")
        
        try:
            run_kernel_telemetry_analysis(
                malware_path=full_path,
                malware_name=filename,
                execution_time=execution_time
            )
            
            successful += 1
            log_success(f"Analysis completed: {family}/{filename}")
            
        except Exception as e:
            failed += 1
            failed_samples.append({
                'family': family,
                'filename': filename,
                'error': str(e)
            })
            log_error(f"Analysis failed: {family}/{filename} - {e}")
            
            # Recovery
            try:
                log_info("Attempting VM recovery...")
                shutdown_and_restore(SNAPSHOT_NO)
                time.sleep(5)
            except:
                pass
        
        # Progress
        elapsed = time.time() - start_time
        remaining = ((elapsed / idx) * len(selected_files)) - elapsed if idx > 0 else 0
        print(f"Progress: {idx}/{len(selected_files)} | Elapsed: {elapsed/60:.1f}m | Remaining: {remaining/60:.1f}m")
        
        # Delay
        if idx < len(selected_files):
            time.sleep(30)
    
    # Summary
    total_time = time.time() - start_time
    success_rate = (successful / len(selected_files)) * 100
    
    print(f"\n📊 ANALYSIS COMPLETE!")
    print("=" * 50)
    print(f"Total samples: {len(selected_files)}")
    print(f"Successful: {successful}")
    print(f"Failed: {failed}")
    print(f"Success rate: {success_rate:.1f}%")
    print(f"Total time: {total_time/60:.1f} minutes")
    
    if failed_samples:
        print(f"\nFailed samples:")
        for sample in failed_samples:
            print(f"  - {sample['family']}/{sample['filename']}: {sample['error']}")

def quick_analysis():
    """Analisis cepat dengan sample minimal"""
    print("\n⚡ QUICK ANALYSIS")
    print("=" * 50)
    print("This will analyze 1 sample from 3 priority families")
    print("Estimated time: ~10 minutes")
    
    confirm = input("\nProceed with quick analysis? (y/n): ").lower()
    if confirm != 'y':
        print("Quick analysis cancelled")
        return
    
    try:
        # Scan and select files
        start_vm_clean()
        
        selected_files = []
        priority_families = PRIORITY_FAMILIES[:3]  # Top 3
        
        for directory in MALWARE_DIRECTORIES:
            family = os.path.basename(directory.replace('\\', '/'))
            if family in priority_families:
                files = scan_vm_directory(directory)
                if files:
                    selected_files.append(files[0])  # Take first file
                    print(f"Selected: {family}/{files[0]['filename']}")
        
        shutdown_vm_clean()
        
        if selected_files:
            run_batch_analysis(selected_files, execution_time=30)
        else:
            log_error("No files found for quick analysis")
            
    except Exception as e:
        log_error(f"Quick analysis failed: {e}")

def custom_analysis():
    """Analisis custom dengan pilihan user"""
    print("\n🎯 CUSTOM ANALYSIS")
    print("=" * 50)
    
    # Pilih families
    print("Available families:")
    for i, family in enumerate(PRIORITY_FAMILIES, 1):
        print(f"{i}. {family}")
    
    family_input = input(f"\nSelect families (comma-separated numbers, e.g. 1,2,3): ").strip()
    
    try:
        family_indices = [int(x.strip()) - 1 for x in family_input.split(',')]
        selected_families = [PRIORITY_FAMILIES[i] for i in family_indices if 0 <= i < len(PRIORITY_FAMILIES)]
    except:
        log_error("Invalid family selection")
        return
    
    if not selected_families:
        log_error("No families selected")
        return
    
    # Pilih jumlah samples
    try:
        samples_per_family = int(input("Samples per family (1-10): "))
        if samples_per_family < 1 or samples_per_family > 10:
            samples_per_family = 3
    except:
        samples_per_family = 3
    
    # Pilih execution time
    try:
        execution_time = int(input("Execution time per sample in seconds (30-120): "))
        if execution_time < 30 or execution_time > 120:
            execution_time = 60
    except:
        execution_time = 60
    
    print(f"\nCustom Analysis Configuration:")
    print(f"Families: {selected_families}")
    print(f"Samples per family: {samples_per_family}")
    print(f"Execution time: {execution_time}s")
    print(f"Estimated total samples: {len(selected_families) * samples_per_family}")
    print(f"Estimated time: {(len(selected_families) * samples_per_family * 2):.0f} minutes")
    
    confirm = input("\nProceed with custom analysis? (y/n): ").lower()
    if confirm != 'y':
        print("Custom analysis cancelled")
        return
    
    try:
        # Scan and select files
        start_vm_clean()
        
        selected_files = []
        
        for directory in MALWARE_DIRECTORIES:
            family = os.path.basename(directory.replace('\\', '/'))
            if family in selected_families:
                files = scan_vm_directory(directory)
                selected_files.extend(files[:samples_per_family])
                print(f"Selected {len(files[:samples_per_family])} files from {family}")
        
        shutdown_vm_clean()
        
        if selected_files:
            run_batch_analysis(selected_files, execution_time)
        else:
            log_error("No files found for custom analysis")
            
    except Exception as e:
        log_error(f"Custom analysis failed: {e}")

# =================================================================================================
# INTERACTIVE MENU
# =================================================================================================

def show_menu():
    """Show main menu"""
    print("\n" + "="*80)
    print("🔬 INTERACTIVE BATCH MALWARE ANALYSIS SYSTEM")
    print("="*80)
    print(f"VM: {VM_NAME}")
    print(f"Snapshot: {SNAPSHOT_NO}")
    print(f"Available families: {len(MALWARE_DIRECTORIES)}")
    print("="*80)
    
    print("\n🚀 PILIHAN MENU:")
    print("1. Test VM Connection         - Test koneksi ke VM")
    print("2. Scan Available Malware     - Scan semua malware yang tersedia")
    print("3. Quick Analysis (Fast)      - Analisis cepat 3 families")
    print("4. Custom Analysis            - Pilih families dan settings sendiri")
    print("5. Check Results              - Lihat hasil analisis sebelumnya")
    print("6. Exit                       - Keluar")
    
    print("\n💡 REKOMENDASI: Mulai dengan pilihan 1, lalu 2, baru 3 atau 4")

def check_previous_results():
    """Check hasil analisis sebelumnya"""
    print("\n📊 CHECKING PREVIOUS RESULTS...")
    print("=" * 50)
    
    results_folder = 'kernel-telemetry-results'
    found_results = False
    
    if os.path.exists(results_folder):
        files = os.listdir(results_folder)
        csv_files = [f for f in files if f.endswith('.csv')]
        
        if csv_files:
            print(f"Found {len(csv_files)} CSV result files:")
            for f in csv_files:
                file_path = os.path.join(results_folder, f)
                size = os.path.getsize(file_path)
                modified = datetime.fromtimestamp(os.path.getmtime(file_path))
                print(f"  - {f} ({size} bytes, {modified.strftime('%Y-%m-%d %H:%M')})")
            found_results = True
    
    if not found_results:
        print("No previous results found")
    
    return found_results

def main_menu():
    """Main interactive menu loop"""
    while True:
        show_menu()
        
        try:
            choice = input("\n➤ Pilih menu (1-6): ").strip()
            
            if choice == '1':
                test_vm_connection()
                
            elif choice == '2':
                files = scan_all_malware()
                if files:
                    print(f"\nScan complete! Found {len(files)} malware files total.")
                
            elif choice == '3':
                quick_analysis()
                
            elif choice == '4':
                custom_analysis()
                
            elif choice == '5':
                check_previous_results()
                
            elif choice == '6':
                print("\n👋 Goodbye!")
                break
                
            else:
                print("❌ Invalid choice! Please select 1-6")
                
        except KeyboardInterrupt:
            print("\n\n👋 Interrupted by user. Goodbye!")
            break
        except Exception as e:
            log_error(f"Unexpected error: {e}")
        
        input("\nPress Enter to continue...")

# =================================================================================================
# START THE INTERACTIVE SYSTEM
# =================================================================================================

print("🚀 INTERACTIVE BATCH MALWARE ANALYSIS SYSTEM LOADED!")
print("="*60)
print("Ready to use! Starting interactive menu...")
print("="*60)

# Auto-start the menu
main_menu()

🚀 INTERACTIVE BATCH MALWARE ANALYSIS SYSTEM LOADED!
Ready to use! Starting interactive menu...

🔬 INTERACTIVE BATCH MALWARE ANALYSIS SYSTEM
VM: windows11-clone Clone
Snapshot: 34
Available families: 40

🚀 PILIHAN MENU:
1. Test VM Connection         - Test koneksi ke VM
2. Scan Available Malware     - Scan semua malware yang tersedia
3. Quick Analysis (Fast)      - Analisis cepat 3 families
4. Custom Analysis            - Pilih families dan settings sendiri
5. Check Results              - Lihat hasil analisis sebelumnya
6. Exit                       - Keluar

💡 REKOMENDASI: Mulai dengan pilihan 1, lalu 2, baru 3 atau 4

📁 SCANNING ALL MALWARE...
[14:58:39] Starting VM...
[14:58:39] ❌ Scan failed: name 'restore' is not defined
[14:58:39] Shutting down VM...

🔬 INTERACTIVE BATCH MALWARE ANALYSIS SYSTEM
VM: windows11-clone Clone
Snapshot: 34
Available families: 40

🚀 PILIHAN MENU:
1. Test VM Connection         - Test koneksi ke VM
2. Scan Available Malware     - Scan semua malware yang ter

## Manual Operations

Use these cells to perform individual operations if needed for testing or debugging.

In [ ]:
def check_vm_status():
    """Check if VM is running."""
    try:
        output = subprocess.check_output(f'vboxmanage showvminfo "{vm_name}" | grep State', shell=True).decode()
        print(output.strip())
    except:
        print("Error checking VM status")

def manual_restore():
    """Manually restore VM to snapshot 31."""
    restore(snapshot_no)

def manual_start():
    """Manually start the VM."""
    start_vm()
    
def manual_shutdown():
    """Manually shutdown the VM."""
    print(f"[{tstamp()}] Powering off VM")
    try:
        check_output_t(f'vboxmanage controlvm "{vm_name}" poweroff')
        print(f"[{tstamp()}] Powering off VM done")
    except:
        print('VM (probably) already off')

def manual_install_driver():
    """Manually install kernel driver."""
    install_kernel_driver()

def manual_start_driver():
    """Manually start GP driver."""
    start_gp_driver()

def manual_process_telemetry():
    """Manually process telemetry data."""
    process_telemetry_data()

def manual_copy_results():
    """Manually copy CSV results."""
    copy_csv_results()

In [ ]:
# Check VM status
# check_vm_status()

In [ ]:
# Manually restore VM to snapshot 31
# manual_restore()

In [ ]:
# Manually start VM
# manual_start()

In [ ]:
# Manually install kernel driver
# manual_install_driver()

In [ ]:
# Manually start GP driver service
# manual_start_driver()

In [ ]:
# Manually process telemetry data
# manual_process_telemetry()

In [ ]:
# Manually copy CSV results
# manual_copy_results()

In [ ]:
# Manually shutdown VM
# manual_shutdown()

## Debug and Testing Functions

Use these functions for debugging individual components.

In [ ]:
def test_vm_communication():
    """Test basic communication with VM."""
    print("Testing VM communication...")
    result = run_vm_command("echo Hello from VM!")
    if result:
        print(f"VM Response: {result.stdout}")
    else:
        print("VM communication failed")

def test_driver_status():
    """Check GP driver service status."""
    print("Checking GP driver status...")
    result = run_vm_command("sc query gp-driver")
    if result:
        print(f"Driver Status: {result.stdout}")

def list_hafidz_directory():
    """List contents of Hafidz directory."""
    print("Listing Hafidz directory contents...")
    result = run_vm_command(f"dir {hafidz_dir}")
    if result:
        print(f"Directory contents:\n{result.stdout}")

def list_desktop_files():
    """List files on desktop to see available malware samples."""
    print("Listing desktop files...")
    result = run_vm_command("dir C:\\Users\\wineleven\\Desktop\\*.exe")
    if result:
        print(f"Available executables:\n{result.stdout}")

In [ ]:
# Test VM communication
# test_vm_communication()

In [ ]:
# Test driver status
# test_driver_status()

In [ ]:
# List Hafidz directory
# list_hafidz_directory()

In [ ]:
# List available malware samples on desktop
# list_desktop_files()